# Tidy Data in Python
The examples and code in this notebook are made by [Jean-Nicholas Hould](http://www.jeannicholashould.com/)

Detailed explanations for important code snippets are provided by Mervat Abuelkheir as part of the CSEN1095 Data Engineering Course.

The goal of this notebook is to show how a messy dataset can be tidied into proper rows representing objects, columns representing attributes, and cells representing scalar values.

Pay attention to the <span style="color:red"> <b> paragraphs in bold red</b></span>; they ask you to do something and provide input!

First thing we need to do is import some libraries.

In [1]:
import pandas as pd
import datetime # to handle date/time attributes
from os import listdir # os is a module for interacting with the OS
from os.path import isfile, join # to verify file object, and concatenate paths
import glob # to find pathnames matching a specific pattern
import re # regular expressions :)

## Examining the datasets

In this part of the exercise we will import a number of datasets and examine their structure to verify if the datasets are tidy.

Remember the requirements for a tidy dataset:
<br> 1- Each row describes a single object
<br> 2- Each column describes a property/attribute of that object
<br> 3- Column values have the same measurement unit
<br> 4- Columns contain atomic/scalar values (no multiple values per table cell)

For each dataset imported, test your ability to identify is it is tidy or not.

### Dataset 1: Pew Research Center

Pew Research Center is a famous center in the US that performs polling surveys on citizens. This is example data about the breakdown of yearly income per religion.

In [ ]:
df = pd.read_csv("./data/pew-raw.csv")
df

<span style="color:red"> <b> What are the attributes of interest? How are they organized? Is the dataset tidy? </b></span> 
    
You can brainstorm your thought process and document in a new cell if you like.
<br>Instructions for beginners:
<br>- Add a new cell from the notebook menu above (+ button).
<br>- Double click anywhere inside the new cell to enter edit mode.
<br>- When done, press CTRL+ENTER or SHIFT+ENTER to commit content.
<br>- You can edit content anytime by double clicking inside the cell.

## Let's tidy the dataset!

The melt function is used to change the format of a pandas data frame from wide to long, assigning one column as an identifier and "unpivoting" the others.

In [ ]:
pd.melt?

In [ ]:
# melt method takes as input a dataframe, one or more identifier attributes, one or more attribute names, and value attribute 
# define new pandas dataframe, religion column will be identifier attribute
# values spread across multiple column headers of income ranges will be unpivoted into new attribute "income"
# actual frequencies of citizens with specific income range will be unpivoted into new attribute "freq"
formatted_df = pd.melt(df,["religion"], var_name="income", value_name="freq")
formatted_df = formatted_df.sort_values(by=["religion"]) # just sorting the new table by religion attribute
formatted_df.head(10) # show first 10 rows

In [ ]:
formatted_df.tail(5)

<span style="color:red"> <b> Why do the indices that are added automatically by pandas appear out of order? </b></span> 
<br>(Just a question to let you think of how pandas dataframes are indexed.)

### Dataset 2: Billboard Top 100

This dataset outlines data about the top hit songs on the Billboard list. 

In [ ]:
df = pd.read_csv("./data/billboard.csv", encoding="mac_latin2")
df.head(10)

<span style="color:red"> <b> Again: What are the attributes of interest? How are they organized? Is the dataset tidy? </b></span>

The structure of the dataset is more complex than the previous one, and it is not immediately clear what a typical row should represent or look like. Answering the above questions helps you frame the data better. 
<br>You can brainstorm your thought process and document in a new cell if you like.

## Let's tidy the dataset!

One way a record could be organized is to make it represent the rank of each song in every week the song was on the Billboard list. This omits the need to keep track of all 76 weeks data, which is null for most of the songs.

A record would have data about the year, artist, track, time, genre, week, rank, and date.

The unique identifier is no single attribute, as one artist can have the track on the billboards at the same year, genre, and time. The only difference would be the week, rank, and date (since date is correlated with week). Therefore, to identify a track's rank and week, we need to use the year, artist, track, time, genre, and date as a combined unique identifier.


### <span style="color:blue"> Note on conversions in Python</span>

<span style="color:blue"> The following conversions are accepted by Python:</span>
<br><span style="color:blue"> - passing a string representation of an integer into int</span>
<br><span style="color:blue"> - passing a string representation of a float into float</span>
<br><span style="color:blue"> - passing a string representation of an integer into float</span>
<br><span style="color:blue"> - passing an integer into float</span>
<br><span style="color:blue"> - passing a float into int</span>

<span style="color:blue"> You get an error if you pass a string representation of a float (or anything other than an integer) into int</span>
<br><span style="color:blue"> This is especially problematic if you have NaN values that are float and you want to convert them to integers. It does not work using int, and you have to use Int32. </span>

Now back to tidying up the Billboard dataset!

In [ ]:
# Melting
# Define unique identifiers in one variable. Include both dates of entry and peak for now; will be merged into one attribute later.
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
# Now melt structure to have identifiers, variable name (week) and values (rank)
df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")
df.head(5)

In [ ]:
# Formatting 
# First, for week attribute, extract week number from string representation of week column names and convert to float then to integer
df["week"] = df["week"].str.extract('(\d+)', expand=False).astype(float).astype(int) 
# Second, extract rank values and convert them to integer
df["rank"] = df["rank"].astype('Int32')
df.head(5)

In [ ]:
# Cleaning out unnecessary rows
df = df.dropna()

# Create "date" columns
# Date for each week is date the track entered the billboard + number of weeks passed for an entry
# Example: if date entered is 26/02/2000, then this is the date for week 1, and the date will change for week 2 to become 04/03/2000, and so on
df["date"] = pd.to_datetime(df["date.entered"]) + pd.to_timedelta(df["week"], unit='w') - pd.DateOffset(weeks=1)

df.head(5)

In [ ]:
# Frame the final tidy data, replacing the dates of entry and peak with only the date, then sort by the identifiers
final_df = df[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]
final_df = final_df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Assigning the tidy dataset to a variable for future usage
billboard = final_df
billboard.head(5)

In [ ]:
billboard.tail(5)

<span style="color:red"><b>Why did we convert the week string to float before converting it to int?</b></span>

<span style="color:red"><b>What does the parameter '(\d+)' in the string.extract method do? </b></span>

In [ ]:
# Now let's check the tidied data frame
# Separating this line of code to avoid running the formatting code multiple times and getting errors
final_df.head(10)

### Dataset 3: Tubercolosis

This dataset outlines the number of tubercolosis patients in different countries in the year 2000.

A few notes on the raw data set:

- The columns starting with "m" or "f" contain multiple variables: 
    - Sex ("m" or "f")
    - Age Group ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mixture of 0s and missing values("NaN"). This is due to the data collection process and the distinction is important for this dataset.

In [ ]:
df = pd.read_csv("./data/tb-raw.csv")
df

<span style="color:red"> <b> Again: What are the attributes of interest? How are they organized? Is the dataset tidy? </b></span>

## Let's tidy the dataset!

Same as what we did before: We need identifiers, we need the column names to represent variables (two in this case, since the column names carry information about gender and age group), and we need the frequency values to be in one column.


In [ ]:
# Let's use the year and country as unique identifiers, and name the # of patients as "cases" and the column variables as "sex and age"
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df.head(3)

In [ ]:
# Extract Sex, Age lower bound and Age upper bound group
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)    

In [ ]:
# tmp_df now has multiple columns corresponding to the strings extracted from the column names. Now name the columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]

# Create "age" column based on "age_lower" and "age_upper"
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

In [ ]:
tmp_df.head(3)

In [ ]:
# Merge - axis parameter indicates the axis along which merge will take place. 1 means by columns
df = pd.concat([df, tmp_df], axis=1)

# Drop unnecessary columns and rows
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
# Drop null values
df = df.dropna()
# Sort rows by all four attributes
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head(10)

<span style="color:red"><b>What does the parameter value "(\D)(\d+)(\d{2})" do?</b></span>

### Dataset 4: Global Historical Climatology Network

In [ ]:
df = pd.read_csv("./data/weather-raw.csv")
df.head(10)

In this dataset, variables are stored in both rows and columns. tmax and tmin stand for max and min temperatures for each day. Date is broken down to three columns, with the day being spread across multiple columns. We need the data to represent min and max temperatures per date.

Notice that the dataset has many missing values.

## Let's tidy the dataset!

Same as what we did before: We need identifiers, we need the column names to represent variables (min and max, and date!), and we need the temperature values to be in two columns.


In [ ]:
# Let's start first by putting the day values in one column. We will not play with min and max temperatures for now
df = pd.melt(df, id_vars=["id", "year","month","element"], var_name="day_raw")
df.head(10)

In [ ]:
# Extracting day
# df["day"] automatically adds a "day" attribute to the df dataframe
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  
df["id"] = "MX17004"
df.head(3)

In [ ]:
# Convert year, month, and day to numeric values
# Notice the use of the lamda function to apply one instruction to multiple inputs
df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))
df.head(3)

In [ ]:
# Let's define a function to create a date from the different columns. 
# Function accepts a row of 3 values as input and returns consolidated date
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

In [ ]:
# Define date attribute, by having the temporary lamda function call the create_date function
df["date"] = df.apply(lambda row: create_date_from_year_month_day(row), axis=1)
# Drop the redundant columns used to compute date
df = df.drop(['year',"month","day", "day_raw"], axis=1)
# Now drop the missing values
df = df.dropna()

In [ ]:
df.head(3)

In [ ]:
df.pivot_table?

In [ ]:
# Unmelting column "element"
df = df.pivot_table(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df

## <span style="color:red"> Exercise your tidying muscles! </span>

<span style="color:red"><b> The GapMinder dataset includes information about the life expectancy, the GDP per capita, and the population of various countries between the years 1952 and 2007.</b></span>

<span style="color:red"> <b>Import the dataset, investigate it to identify what the potential attributes should be, the problems with the current structure, and think of how to tidy the dataset, and then proceed to tidy the dataset.</b></span>

In [2]:
df = pd.read_csv("./data/gapminder.csv") 
df.head(3)

,continent,country,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,...,pop_1962,pop_1967,pop_1972,pop_1977,pop_1982,pop_1987,pop_1992,pop_1997,pop_2002,pop_2007
0,Africa,Algeria,2449.008185,3013.976023,2550.816880,3246.991771,4182.663766,4910.416756,5745.160213,5681.358539,...,11000948.0,12760499.0,14760787.0,17152804.0,20033753.0,23254956.0,26298373.0,29072015.0,31287142,33333216
1,Africa,Angola,3520.610273,3827.940465,4269.276742,5522.776375,5473.288005,3008.647355,2756.953672,2430.208311,...,4826015.0,5247469.0,5894858.0,6162675.0,7016384.0,7874230.0,8735988.0,9875024.0,10866106,12420476
2,Africa,Benin,1062.752200,959.601080,949.499064,1035.831411,1085.796879,1029.161251,1277.897616,1225.856010,...,2151895.0,2427334.0,2761407.0,3168267.0,3641603.0,4243788.0,4981671.0,6066080.0,7026113,8078314


In [3]:
new_cols = ['gdpPercap', 'lifeExp', 'pop']
col_dict = {
    # KayAI Check
    'gdpPercap': None, 
    'lifeExp': None, 
    'pop': None
} 

In [4]:
for col in new_cols:
    col_dict[col] = pd.concat([df.loc[:, 'continent':'country'], 
                               df.loc[:, f'{col}_1952':f'{col}_2007']], axis=1) 
    col_dict[col] = col_dict[col].melt(id_vars=["continent", "country"], 
                                       var_name="year", value_name=f'{col}')
    col_dict[col]['year'] = col_dict[col]['year'].str.replace(r'\D', '')

In [5]:
new_gapminder_df = pd.concat([col_dict[new_cols[0]], 
                              col_dict[new_cols[1]][new_cols[1]], 
                              col_dict[new_cols[2]][new_cols[2]]], axis=1)

In [6]:
new_gapminder_df

,continent,country,year,gdpPercap,lifeExp,pop
0,Africa,Algeria,1952,2449.008185,43.077,9279525.0
1,Africa,Angola,1952,3520.610273,30.015,4232095.0
2,Africa,Benin,1952,1062.752200,38.223,1738315.0
3,Africa,Botswana,1952,851.241141,47.622,442308.0
4,Africa,Burkina Faso,1952,543.255241,31.975,4469979.0
...,...,...,...,...,...,...
1699,Europe,Switzerland,2007,37506.419070,81.701,7554661.0
1700,Europe,Turkey,2007,8458.276384,71.777,71158647.0
1701,Europe,United Kingdom,2007,33203.261280,79.425,60776238.0
1702,Oceania,Australia,2007,34435.367440,81.235,20434176.0
